In [ ]:
KUBECONFIG_PATH = "<kubectl_config_path>"
SERVICEWEAVER_POD_LABEL = "<service_name>" # eg. "serviceweaver/app=server.out"

In [3]:
from kubernetes import client,config
config.load_kube_config(KUBECONFIG_PATH)
api_instance=client.CoreV1Api()

In [4]:
from uuid import uuid4
debug_container_name=f"ssh-debugger{uuid4()}"
debug_container_name

'ssh-debugger9a935caa-9cc7-4ff7-96fe-758717c90ae6'

In [5]:
def setup_debug_container(pod_name: str, namespace: str = "default") -> None:

    debug_container = client.V1EphemeralContainer(
                    name=debug_container_name,
                    image="h21565897/distributeddebugger:144",
                    target_container_name="serviceweaver",
                    image_pull_policy="Always",
                    stdin=True,
                    tty=True,
                    security_context=client.V1SecurityContext(
                        privileged=True
                    )
                )
    patch_body = {
                "spec": {
                    "ephemeralContainers": [
                        debug_container
                    ]
                }
            }
            
            # Apply the patch
    api_instance.patch_namespaced_pod_ephemeralcontainers(
        name=pod_name,
        namespace="default",
        body=patch_body
    )

In [6]:
pods = api_instance.list_namespaced_pod(
            namespace="default", label_selector=SERVICEWEAVER_POD_LABEL)
len(pods.items)

3

In [7]:
for pod in pods.items:
    setup_debug_container(pod.metadata.name)
    print(f"Debug container injected for pod {pod.metadata.name}")

Debug container injected for pod weaver-main-5bb81ff6-e4a25d35-66497655df-pxt59
Debug container injected for pod weaver-mediarequesthandler-5bb81ff6-5be2b6d6-654948d669-j5s7l
Debug container injected for pod weaver-statusrequesthandler-5bb81ff6-0afde9d8-6d8bbf76b4-5pqrg
